# SLIM MODEL
Since the KNN_Item_CF is the one that perfomed the best so far, now i'm trying a SLIM Model, which is basically the same algorithm but Machine Learning based. It uses the infos from the training data instead of some heuristic (such as cosine similarity ecc...)
https://notebook.community/trangel/Insight-Data-Science/general-docs/recommendation-validation/.ipynb_checkpoints/recommender_systems-validation-checkpoint

# Data loading 
Next cells are used to load the data we need.

In [1]:
import pandas as pd
import numpy as np 
import scipy.sparse as sps
import os
import sys
import matplotlib.pyplot as plt
sys.path.append('..')


In [2]:
#data which contains users (row), items(col) and implicit interaction (data)
dataset = pd.read_csv('data_train.csv')
dataset

,row,col,data
0,0,10080,1.0
1,0,19467,1.0
2,1,2665,1.0
3,1,7494,1.0
4,1,17068,1.0
...,...,...,...
113263,7945,2476,1.0
113264,7945,12319,1.0
113265,7945,21384,1.0
113266,7946,8699,1.0


# Data pre-processing
Pre-processing of data to check for missing users, items ecc...

In [4]:
users = dataset.row
items = dataset.col
data = dataset.data
URM_all = sps.coo_matrix((data, (users, items)))
URM_all = URM_all.tocsr() #fast row access -> fast access to users 
URM_all.shape

(7947, 25975)

In [9]:
ICM_df = pd.read_csv('data_ICM_title_abstract.csv')
ICM_df

,row,col,data
0,0,1185,1.015524
1,0,2507,0.459024
2,0,3534,0.227742
3,0,8766,0.501549
4,0,10862,0.297011
...,...,...,...
490686,25974,12554,0.963016
490687,25974,13003,0.104613
490688,25974,16236,0.118760
490689,25974,18797,0.363301


In [11]:
items = ICM_df.row
features = ICM_df.col
data = ICM_df.data
ICM_all = sps.coo_matrix((data, (items, features)))
ICM_all = ICM_all.tocsr() #fast row access -> fast access to users 
ICM_all.shape

(25975, 20000)

In [26]:
from Base.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

#splitting data into train and validation (from the previous training data)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.85)

In [27]:
#evaluator for validation (used for hyperparameter tuning)
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

In [31]:
#bayes optimizer
from ParameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from MatrixFactorization.PyTorch.MF_MSE_PyTorch import MF_MSE_PyTorch


recommender_class = MF_MSE_PyTorch


parameterSearch = SearchBayesianSkopt(recommender_class,
                                 evaluator_validation=evaluator_validation)

In [32]:
from ParameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from skopt.space import Real, Integer, Categorical
epochs = 200
hyperparameters_range_dictionary = {}
hyperparameters_range_dictionary["batch_size"] = Integer(64,512)
hyperparameters_range_dictionary["num_factors"] = Integer(0, 1000)
hyperparameters_range_dictionary["learning_rate"] = Real(0.0001, 0.1)



    
    
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train],
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)


output_folder_path = "result_experiments/"

import os

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

In [33]:
n_cases = 2
metric_to_optimize = "MAP"

parameterSearch.search(recommender_input_args,
                       parameter_search_space = hyperparameters_range_dictionary,
                       n_cases = n_cases,
                       n_random_starts = 1,
                       save_model = "no",
                       output_folder_path = output_folder_path,
                       output_file_name_root = recommender_class.RECOMMENDER_NAME,
                       metric_to_optimize = metric_to_optimize
                      )

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Config 0 Exception. Config: {'batch_size': 328, 'num_factors': 635, 'learning_rate': 0.04796867684388662} - Exception: Traceback (most recent call last):
  File "..\ParameterTuning\SearchAbstractClass.py", line 362, in _objective_function
    result_dict, result_string, recommender_instance, train_time, evaluation_time = self._evaluate_on_validation(current_fit_parameters_dict)
  File "..\ParameterTuning\SearchAbstractClass.py", line 267, in _evaluate_on_validation
    recommender_instance, train_time = self._fit_model(current_fit_parameters)
  File "..\ParameterTuning\SearchAbstractClass.py", line 248, in _fit_model
    recommender_instance = self.recommender_class(*self.recommender_input_args.CONSTRUCTOR_POSITIONAL_ARGS,
  File "..\MatrixFactorization\PyTorch\MF_MSE_PyTorch.py", line 35, in __init__
    super(MF_MSE_PyTorch, self).__init__()
TypeError: __init__() missing 1 required positional argument:

Traceback (most recent call last):
  File "..\ParameterTuning\SearchAbstractClass.py", line 362, in _objective_function
    result_dict, result_string, recommender_instance, train_time, evaluation_time = self._evaluate_on_validation(current_fit_parameters_dict)
  File "..\ParameterTuning\SearchAbstractClass.py", line 267, in _evaluate_on_validation
    recommender_instance, train_time = self._fit_model(current_fit_parameters)
  File "..\ParameterTuning\SearchAbstractClass.py", line 248, in _fit_model
    recommender_instance = self.recommender_class(*self.recommender_input_args.CONSTRUCTOR_POSITIONAL_ARGS,
  File "..\MatrixFactorization\PyTorch\MF_MSE_PyTorch.py", line 35, in __init__
    super(MF_MSE_PyTorch, self).__init__()
TypeError: __init__() missing 1 required positional argument: 'URM_train'


ValueError: array must not contain infs or NaNs

In [21]:
evaluator_validation.evaluateRecommender(recommender)

EvaluatorHoldout: Processed 5013 ( 100.00% ) in 5.34 sec. Users per second: 938


({10: {'ROC_AUC': 0.08678246875583798,
   'PRECISION': 0.017175344105326087,
   'PRECISION_RECALL_MIN_DEN': 0.05852917335562448,
   'RECALL': 0.05624966385938946,
   'MAP': 0.0271010371145377,
   'MRR': 0.061978380021467966,
   'NDCG': 0.04133352056315101,
   'F1': 0.0263154845841679,
   'HIT_RATE': 0.1717534410532615,
   'ARHR': 0.06879050975400466,
   'NOVELTY': 0.004094948590094968,
   'AVERAGE_POPULARITY': 0.24417093065628168,
   'DIVERSITY_MEAN_INTER_LIST': 0.8900049018601118,
   'DIVERSITY_HERFINDAHL': 0.9889827362482122,
   'COVERAGE_ITEM': 0.03283926852743022,
   'COVERAGE_ITEM_CORRECT': 0.008123195380173243,
   'COVERAGE_USER': 0.6308040770101925,
   'COVERAGE_USER_CORRECT': 0.0861960488234554,
   'DIVERSITY_GINI': 0.005042609879542631,
   'SHANNON_ENTROPY': 7.334267754183512}},
 'CUTOFF: 10 - ROC_AUC: 0.0867825, PRECISION: 0.0171753, PRECISION_RECALL_MIN_DEN: 0.0585292, RECALL: 0.0562497, MAP: 0.0271010, MRR: 0.0619784, NDCG: 0.0413335, F1: 0.0263155, HIT_RATE: 0.1717534, ARH

In [92]:
test_users = pd.read_csv('data_target_users_test.csv')
test_users

,user_id
0,0
1,1
2,2
3,3
4,4
...,...
7939,7942
7940,7943
7941,7944
7942,7945


In [94]:
recommender_alpha = P3alphaRecommender(URM_all)
recommender_alpha.fit(topK=475, alpha = 0.45, implicit = True)

recommender_alpha_ICM = P3alphaRecommender(ICM_all.T)
recommender_alpha_ICM.fit(topK=175, alpha = 0.45)
recommender_alpha_ICM.URM_train = URM_train

hybridrecommender = ItemKNNSimilarityHybridRecommender(URM_all, recommender_alpha_ICM.W_sparse, recommender_alpha.W_sparse)
hybridrecommender.fit(topK=600, alpha = 0.45)

user_id = test_users['user_id']
recommendations = hybridrecommender.recommend(user_id,cutoff = 10)


P3alphaRecommender: URM Detected 1079 (4.15 %) cold items.
P3alphaRecommender: URM Detected 2 (0.01 %) cold users.
P3alphaRecommender: URM Detected 25 (0.10 %) cold items.
ItemKNNSimilarityHybridRecommender: URM Detected 1079 (4.15 %) cold items.


In [95]:
for index in range(len(recommendations)):
    #print(element)
    recommendations[index]=np.array(recommendations[index])
    #print(type(element))
print(len(recommendations))

7944


In [96]:
test_users['item_list']= recommendations
#test_users['item_list'] =  test_users['item_list'].apply(lambda x: x.replace('[','').replace(']','')) 
test_users['item_list'] = pd.DataFrame([str(line).strip('[').strip(']').replace("'","") for line in test_users['item_list']])

#convert the string columns to int
#test_users['item_list'] = test_users['item_list'].astype(int)
test_users
test_users.to_csv('submission.csv', index=False)
